<a href="https://colab.research.google.com/github/jko6422/testfile/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # Minst 시험


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
cd /content/drive/MyDrive/colab/Minst_test/

In [ ]:
#1. 데이터 세트 만들기(오픈소스로 테스트중입니다.)

#필요 함수 import
import cv2 
import numpy as np
from google.colab.patches import cv2_imshow

#이미지 파일 경로 지정. 이미지는 camscanner로 보정 후 드라이브에 업로드.
filename = '/content/drive/MyDrive/colab/Minst_test/data/testfile.jpg'

#그레이 스케일로 변환 
src = cv2.imread(filename,cv2.IMREAD_GRAYSCALE) 

#이미지의 크기를 5분의 1로 줄여서 확인 
src = cv2.resize(src , (int(src.shape[1]/5),int(src.shape[0]/5))) 

#줄여진 이미지 확인
#cv2_imshow(src) 
#k = cv2.waitKey(0) 
#cv2.destroyAllWindows()

#양 옆의 노이즈를 제거
src = src[0:src.shape[0]-10, 15:src.shape[1]-25]

#영상 이진화 
ret , binary = cv2.threshold(src,170,255,cv2.THRESH_BINARY_INV) 

#이진화 이미지 확인
#cv2_imshow(binary) 
#k = cv2.waitKey(0) 
#cv2.destroyAllWindows()

#노이즈 제거(camscanner에 의해 어느 정도 보정이 되었으므로 각주 처리)
#binary = cv2.morphologyEx(binary , cv2.MORPH_OPEN , cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2,2)), iterations = 2) 

#노이즈 제거 이미지 확인
#cv2_imshow(binary) 
#k = cv2.waitKey(0) 
#cv2.destroyAllWindows()

#외곽선 검출
contours , hierarchy = cv2.findContours(binary , cv2.RETR_EXTERNAL , cv2.CHAIN_APPROX_SIMPLE)
 
#이진화 이미지를 color이미지로 복사(확인용) 
color = cv2.cvtColor(binary, cv2.COLOR_GRAY2BGR)

#초록색으로 외곽선을 그려준다.
cv2.drawContours(color , contours , -1 , (0,255,0),3)

#리스트 연산을 위해 초기변수 선언 
bR_arr = [] 
digit_arr = [] 
digit_arr2 = [] 

# 검출한 외곽선에 사각형을 그려서 배열에 추가. 이때 크기가 작은 노이즈는 1차로 거른다.
# boundingRect : 외각선을 감싸는 최소 크기 사각형 반환. x, y는 좌표. w는 폭, h는 넓이.
# append : 리스트에 요소 추가
for i in range(len(contours)) : 
  bin_tmp = binary.copy() 
  x,y,w,h = cv2.boundingRect(contours[i]) 
  if w and h > 5 : 
      bR_arr.append([x,y,w,h])

#x축을 기준으로 정렬
bR_arr = sorted(bR_arr, key=lambda num : num[0], reverse = False)
print(bR_arr[:5])
print(len(bR_arr))

#데이터에 사각형그리기. 10개씩 리스트로 묶어서 저장 
#순서는 !부터 위에서 아래로 10번(count). 그 뒤에 다음 이모티콘을 저장하는 방식.(county)
count = 0
county = 0

for x,y,w,h in bR_arr : 
  tmp_y = bin_tmp[y-2:y+h+2,x-2:x+w+2].shape[0] #y축 계산
  tmp_x = bin_tmp[y-2:y+h+2,x-2:x+w+2].shape[1] #x축 계산
  # %, !, ?는 크기를 넓게 세팅.
  # ^ 와 * 은 크기가 작으므로 조건없이 별도로 처리하게 세팅.
  # 그 이외는 글자 크기에 맞게 세팅.
  if (tmp_x and tmp_y > 20) or county == 5 or county == 7: 
      if county == 0 or county == 8: 
          if x < 0 :
              cv2.rectangle(color,(0,y-4),(x+w+2,y+h+24),(0,0,255),1) 
              digit_arr.append(bin_tmp[y-2:y+h+24,0:x+w+2]) 
          else : 
              cv2.rectangle(color,(x-6,y-4),(x+w+2,y+h+24),(0,0,255),1) 
              digit_arr.append(bin_tmp[y-4:y+h+24,x-6:x+w+2]) 
      elif county == 4 : 
          cv2.rectangle(color,(x-8,y-6),(x+w+4,y+h+8),(0,0,255),1) 
          digit_arr.append(bin_tmp[y-6:y+h+8,x-8:x+w+4]) 
      else : 
          cv2.rectangle(color,(x-2,y-2),(x+w+2,y+h+2),(0,0,255),1) 
          digit_arr.append(bin_tmp[y-2:y+h+2,x-2:x+w+2]) 
      
      count += 1

      if count == 10 : 
          digit_arr2.append(digit_arr) 
          digit_arr = [] 
          count = 0
          county += 1

#결과 확인
cv2_imshow(color) 
k = cv2.waitKey(0) 
cv2.destroyAllWindows()

In [ ]:
#리스트에 저장된 이미지를 32x32의 크기로 리사이즈해서 순서대로 저장 
for i in range(0,len(digit_arr2)) : 
  for j in range(len(digit_arr2[i])) :
     
    # !과 ?의 경우 비율 유지를 위한 마스크 생성
     if i == 0 or i==8: 
         width = digit_arr2[i][j].shape[1]
         height = digit_arr2[i][j].shape[0] 
         tmp = (height - width)/2 
         mask = np.zeros((height,height)) 
         mask[0:height,int(tmp):int(tmp)+width] = digit_arr2[i][j]
         digit_arr2[i][j] = cv2.resize(mask,(32,32)) 
     else:
         digit_arr2[i][j] = cv2.resize(digit_arr2[i][j],(32,32))

for i in range(0,len(digit_arr2)) : 
  for j in range(len(digit_arr2[i])) :
      cv2.imwrite('/content/drive/MyDrive/colab/Minst_test/data/test1/testfile'+str(i+1)+'_'+str(j+1)+'.png',digit_arr2[i][j])

In [ ]:
#2. 데이터 학습(아직 데이터 셋이 완성되지 않았으므로 mnist로 테스트.)

#데이터 갯수 설정, mnist import
from keras.datasets import mnist
(X_train, Y_class_train), (X_test, Y_class_test) = mnist.load_data()
print("학습셋 이미지 수 : %d 개" % (X_train.shape[0]))
print("테스트셋 이미지 수 : %d 개" % (X_test.shape[0]))

# 이미지 형태 출력을 위한 pyplot 모듈 import
import sys
import matplotlib.pyplot as plt

# 위 60000개 데이터 배열에서 0번째 원소를 흑백으로 출력
plt.imshow(X_train[0], cmap='Greys')
plt.show()

# 0번째 원소의 모든 데이터를 출력
for x in X_train[0]:
    for i in x:
        sys.stdout.write('%d  ' % i)
    sys.stdout.write('\n')

# 딥러닝에 필요한 케라스 함수 호출
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense

# 필요 라이브러리 호출
import numpy
import tensorflow as tf

# 데이터 셋 호출
from keras.datasets import mnist

# 실행 시마다 같은 결과값 도출을 위한 시드 설정
numpy.random.seed(0)
tf.random.set_seed(0)

# 데이터를 불러와서 각 변수에 저장
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

# 학습에 적합한 형태로 데이터 가공
X_train = X_train.reshape(X_train.shape[0], 784).astype('float32') / 255
X_test = X_test.reshape(X_test.shape[0], 784).astype('float32') / 255

# 클래스를 학습에 이용하기 위해 데이터 가공
Y_train = np_utils.to_categorical(Y_train, 10)
Y_test = np_utils.to_categorical(Y_test, 10)

# 딥러닝 모델 구조 설정(2개층, 512개의 뉴런 연결, 10개 클래스 출력 뉴런, 784개 픽셀 input 값, relu와 softmax 활성화 함수 이용)
model = Sequential()
model.add(Dense(512, input_dim=784, activation='relu'))
model.add(Dense(10, activation='softmax'))

# 딥러닝 구조 설정(loss 옵션을 다중 클래스에 적합한 categorical_crossentropy, 옵티마이저는 adam 설정)
model.compile(loss='categorical_crossentropy', optimizer='adam',
              metrics=['accuracy'])

# 모델 실행(X_test, Y_test로 검증, 200개씩 30번 학습)
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=30, batch_size=200, verbose=2)

# 학습 정확도, 검증 정확도 출력
print('\nAccuracy: {:.4f}'.format(model.evaluate(X_train, Y_train)[1]))
print('\nVal_Accuracy: {:.4f}'.format(model.evaluate(X_test, Y_test)[1]))

# 모델 저장
model.save('Predict_Model.h5')

In [ ]:
# 3. 딥러닝 결과 확인
# 딥러닝에 필요한 케라스 함수 호출
from keras.models import load_model
from keras.utils import np_utils

# 필요 라이브러리 호출(PIL은 이미지파일 처리위함)
from PIL import Image
import numpy as np

# test.png는 그림판에서 붓으로 숫자 8을 그린 이미지 파일. 이미지가 있는 경로를 아래 url위치에 넣어주세요
# test.png 파일을 열어서 L(256단계 흑백이미지)로 변환
img = Image.open("/content/drive/MyDrive/colab/Minst_test/test.png").convert("L")

# 변환된 이미지 보기
plt.imshow(img)
plt.show

# 이미지를 784개 흑백 픽셀로 사이즈 변환
img = np.resize(img, (1, 784))

# 데이터를 모델에 적용할 수 있도록 가공
test_data = ((np.array(img) / 255) - 1) * -1

# 모델 불러오기
model = load_model('Predict_Model.h5')

# 클래스 예측 함수에 가공된 테스트 데이터 넣어 결과 도출
res = np.argmax(model.predict(test_data), axis=-1)
print(res)